# Steal The Files - Dokumentation (DE)
Client | Server | Core - JavaFX 2D-Plattformer

**Inhalt**
1. &Uuml;berblick &amp; Architektur
2. Setup &amp; Start
3. Spiellogik: Schritte, Entscheidungen, Notizen + Code
4. GUI/Navi: Schritte, Entscheidungen, Notizen + Code
5. Levelaufbau: Schritte, Entscheidungen, Notizen + Code
6. Audio &amp; Einstellungen + Code
7. Netzwerk &amp; Daten + Code
8. Debugging &amp; offene Punkte + Code
9. Roadmap-Nuggets

## 1. &Uuml;berblick &amp; Architektur
- Ziel: USB holen -> Files freischalten -> Folder einsammeln -> Gefahren &uuml;berleben -> Finish ber&uuml;hren.
- Startpfad &amp; Levelwahl:
```java
switch (config.getString("currentLevel")) {
    case "Tutorial" -> setCurrentLevel(new TutorialLevel());
    case "Second"   -> setCurrentLevel(new SecondLevel());
    case "Boss"     -> setCurrentLevel(new BossLevel());
}
```
- Game-Loop (eigener Thread, UI via JavaFX):
```java
while (running) {
    long now = System.nanoTime();
    double delta = (now - lastTime) / 1_000_000_000.0;
    lastTime = now;
    Platform.runLater(() -> currentScreen.update(delta));
    Thread.sleep(5);
}
```

## 2. Setup &amp; Start
- Build: 
```bash
mvn clean install
```
- Start (IDE): Server `de.cyzetlc.hsbi.Server`, Client `de.cyzetlc.hsbi.game.Game`.
- Config-Auszug:
```json
{
  "currentLevel": "Tutorial",
  "soundVolume": 0.6,
  "soundMuted": false,
  "messages": { "game.prefix": "HSBI-GAME" },
  "mysql": {
    "hostname": "db-host",
    "database": "hsbi",
    "username": "user",
    "password": "<secret>",
    "port": 3306,
    "poolSize": 3
  },
  "tutorialFinished": false
}
```

## 3. Spiellogik: Schritte, Entscheidungen, Notizen + Code
**Ablauf (GameScreen)**
- Input/Toggles:
```java
boolean f2 = screenManager.getInputManager().pollJustPressed(KeyCode.F2);
if (f2) { showDebugBar = !showDebugBar; }
```
- Bewegung/Physik (Auszug):
```java
dy += Game.gravity * delta;
double nextX = x + dx;
double nextY = y + dy;
Rectangle2D nextBounds = new Rectangle2D(nextX, nextY, player.getWidth(), player.getHeight());
```
- NoClip/GodMode (F3):
```java
if (f3) { player.setNoClip(!player.isNoClipEnabled()); player.setGodMode(player.isNoClipEnabled()); }
if (player.isNoClipEnabled()) { /* freie Bewegung ohne Gravitation */ return; }
```
- RobotEnemy feuert Laser:
```java
boolean close = Math.abs(player.getLocation().getX() - getLocation().getX()) < 440;
if (close && fireTimer >= fireCooldown) {
    fireTimer = 0;
    return new LaserBlock(new Location(spawnX, eyeY), dir, 320);
}
```
- GasBarrier Interaktion (E + Flipper):
```java
if (block instanceof GasBarrierBlock barrier && interactPressed && player.hasFlipper()) {
    barrier.deactivate();
    continue;
}
```
- Perk-Reset (JumpBoost/SpeedBoost):
```java
PauseTransition delay = new PauseTransition(Duration.seconds(10));
delay.setOnFinished(e -> Game.jumpPower = 800);
delay.play();
```
**Design-Notizen**
- Folder erst nach USB: `canCollectFiles` gate.
- Laser-Lifetime fix (4s) statt Distanz.
- Physik delta-basiert, UI getrennt.
**Technik-Probleme**
- Pause Overlay stoppt Logik nicht.
- `Level.update()` ungenutzt; Kamera ohne Bounds.

## 4. GUI/Navi: Schritte, Entscheidungen, Notizen + Code
**Aufbau**
- LoadingScreen:
```java
Timeline tl = new Timeline(new KeyFrame(Duration.millis(3000), kvWidth));
timeline.setOnFinished(e -> screenManager.showScreen(mainMenu));
```
- SettingsScreen Volume/Mute:
```java
volumeSlider.valueProperty().addListener((o, ov, nv) -> {
    SoundManager.setVolume(nv.doubleValue());
    updateVolumeLabel();
});
muteBtn.setOnAction(e -> {
    SoundManager.setMuted(!SoundManager.isMuted());
    volumeSlider.setDisable(SoundManager.isMuted());
});
```
- GameScreen HUD-Update (Files/Health):
```java
this.totalFolderCount = countFolderBlocks();
filesProgressLbl.setText("Files: " + countActiveFolders() + "/" + totalFolderCount);
updateHealth();
```
- Pause Overlay:
```java
private void togglePause() {
    paused = !paused;
    pauseOverlay.setVisible(paused);
}
```
**Design-Notizen**
- Fullscreen, keine OS-Deko.
- Buttons links oben, Health rechts oben.
**Technik-Probleme**
- LevelFinished zeigt Hardcoded-Stats.
- Kein Resize-Handling.

## 5. Levelaufbau: Schritte, Entscheidungen, Notizen + Code
**TutorialLevel (Ausschnitt)**
```java
platforms.add(new Platform(0, h-300, 450, 600, root));
blocks.add(new USBStickBlock(new Location(200, h-360)));
blocks.add(new FloatingPlatformBlock(new Location(940, h-340), new Location(1224, h-340), 120));
placeLavaBetweenPlatforms(h);
```
**SecondLevel (Ausschnitt)**
```java
blocks.add(new FlipperItem(new Location(220, h-320)));
blocks.add(new RobotEnemyBlock(new Location(5000, h-396), 500, 180));
// blocks.add(new GasBarrierBlock(new Location(4550, h-428), 64, 128));
blocks.add(new FolderBlock(new Location(3600, h-530)));
```
**BossLevel (Ausschnitt)**
```java
blocks.add(new RobotEnemyBlock(new Location(250, h-396), 500, 180));
blocks.add(new RobotEnemyBlock(new Location(1400, h-396), 500, 180));
blocks.add(new FinishBlock(new Location(1800, h-390)));
```
**Design-Notizen**
- USB vor Foldern als Gate.
- Flipper fr&uuml;h im SecondLevel.
**Technik-Probleme**
- Hart codierte Koordinaten, keine JSON-Levels.
- Kamera-Bounds fehlen.

## 6. Audio &amp; Einstellungen + Code
- Volume/Mute Persistenz:
```java
public static void setVolume(double volume) {
    globalVolume = Math.max(0, Math.min(1, volume));
    Game.getInstance().getConfig().getObject().put("soundVolume", volume);
    Game.getInstance().getConfig().save();
}
public static void setMuted(boolean muted) {
    SoundManager.muted = muted;
    Game.getInstance().getConfig().getObject().put("soundMuted", muted);
    Game.getInstance().getConfig().save();
}
```
- Ducking-Beispiel:
```java
SoundManager.playWithDuck(Sound.JUMP_BOOST, 1.0, 0.06);
```
- Hintergrundmusik:
```java
SoundManager.playBackground(Music.MENU, true);
```

## 7. Netzwerk &amp; Daten + Code
- Client Connect:
```java
InetAddress ip = InetAddress.getByName("localhost");
socket = new Socket(ip, 25570);
sendPacket(new ClientLoginPacket(thePlayer.getUuid()));
networkExecutor.execute(new ReceiverTask());
networkExecutor.execute(new SenderTask());
```
- Heartbeat-Sender:
```java
while (!socket.isClosed()) {
    sendPacket(new ClientDataPacket(thePlayer.getUuid()));
    Thread.sleep(50);
}
```
- Server Handler (vereinfacht):
```java
int bytesRead = dis.read(received);
Packet packet = SerializationUtils.deserialize(actualReceivedData, Packet.class);
if (((EventCancelable) new ReceivePacketEvent(packet, socket).call()).isCancelled()) break;
```
- Score senden:
```java
sendPacket(new ClientSubmitScorePacket(thePlayer.getUuid(), score, "Level_1"));
```

## 8. Debugging &amp; offene Punkte + Code
- Toggles (F1/F2/F3):
```java
if (f1) { showTooltips = !showTooltips; }
if (f2) { showDebugBar = !showDebugBar; }
if (f3) { player.setNoClip(!player.isNoClipEnabled()); player.setGodMode(player.isNoClipEnabled()); }
```
- Game Over:
```java
if (player.getHealth() <= 0) { handleGameOver(); return; }
```
- Offenes: Pause=Overlay-only, Level.update() leer, Kamera-Clamps fehlen, GasBarrier im SecondLevel auskommentiert, LevelFinished-Stats Platzhalter.

## 9. Roadmap-Nuggets
- Kurzfristig: echte Pause (Loop stoppen), echte Stats f&uuml;r LevelFinished, Kamera-Bounds, GasBarrier in SecondLevel aktivieren.
- Mittelfristig: Level.update nutzen f&uuml;r mover/timer, Leveldaten aus JSON/Editor, HUD um Timer/Score erweitern.
- Langfristig: Multiplayer/Highscore mit echter DB, Achievements/Serverliste ans Backend binden.